## FCA2 beta, FCastell Auto Analyser
by Fabiano Castello

In [25]:
FC_Auto_Analyser_Version = "FCA2 FC Auto Analyser v0.7 beta (jan/20)"
readme_git = "http://raw.githubusercontent.com/fabianocastello/fca2/master/README.md"

In [26]:
#Parâmetros
datain    = ".\data.in"    #coloque aqui seus arquivos xls ou csv
dataout   = ".\data.out"   #onde analisador vai gravar os resultados
datalog   = ".\data.log"   #onde o analisador vai gravar os logs do processamento 
datatmp   = ".\data.tmp"   #arquivos temporários. Será limpo após o processamento

max_freq  = 10            #numeros de categorias máximas nos campos texto
hist_bins = 10 #qte de bins no histograma 

In [27]:
import os
import sys
import numpy as np
import requests
import pandas as pd
from pandas import ExcelFile 
from datetime import datetime
from datetime import timedelta
import time, socket
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')  # Use '' for auto, or force e.g. to 'en_US.UTF-8'
import seaborn as sns
import matplotlib.pyplot as plt

In [28]:
#Criando diretórios se inexistentes
print("FCA2 Starting")
if not os.path.exists(datatmp):
    print("diretório "+datatmp+" criado")
    os.makedirs(datatmp)
if not os.path.exists(dataout):
    print("diretório "+dataout+" criado")
    os.makedirs(dataout)
if not os.path.exists(datalog):
    print("diretório "+datalog+" criado")
    os.makedirs(datalog)
    
if not os.path.exists(datain):
    print("diretório "+datain+" não existe")
    quit()
    


FCA2 Starting


In [29]:
def repeat_to_length(string_to_expand, length):
    return (string_to_expand * (int(length/len(string_to_expand))+1))[:length]
    
def log_start(log_file_open, message):
    tmp_computer_name = socket.gethostname()
    global log_file ; global log_count; global LogElap; global LogElap_FULL 
    log_file = open(datalog+"/"+log_file_open,"w+")
    log_count = 1  
    now = datetime.now()  
    LogElap = time.time()
    LogElap_FULL = time.time()
    log_file.write("**LOG "+FC_Auto_Analyser_Version+"** \n") 
    log_file.write(repeat_to_length("-",80) + "\n")
    log_file.write(message  + "\n")
    log_file.write(repeat_to_length("-",80) + "\n")
    log_file.write(os.getcwd() + "\n")
    log_file.write(log_file_open + "\n" + "Running on " + tmp_computer_name.upper() + " \n")
    log_file.write(repeat_to_length("-",80) + "\n")
    log_file.write('{:4d}'.format(log_count) + "  " + now.strftime("%d/%m/%Y %H:%M:%S") +
                  "     <<<<< Log start >>>>> \n\n")
    
def log_write(log_message, tela = 0):
    global log_count; global LogElap
    now = datetime.now()
    log_count = log_count + 1
    if tela == 1:
        print('{:4d}'.format(log_count) + "  " + now.strftime("%d/%m/%Y %H:%M:%S") + 
              "    " + log_message + "\n")
    log_file.write('{:4d}'.format(log_count) + "  " + now.strftime("%d/%m/%Y %H:%M:%S") + 
              "     " + log_message + "\n")

    
    LogElap = time.time()
    
def log_close(send_email = False):
    global log_count
    now = datetime.now()
    log_count = log_count + 1
    log_file.write("\n" + '{:4d}'.format(log_count) + "  " + now.strftime("%d/%m/%Y %H:%M:%S") +
                  "     <<<<< Log end >>>>> \n")
    log_file.write(repeat_to_length("-",80) + "\n")
    log_file.write("Running time total:  " + convert(time.time() - LogElap_FULL) + "\n")
    log_file.write(repeat_to_length("-",80) + "\n")
    log_file.write("** Get latest version @ github.com/fabianocastello/fca2\n")

    log_file.close() 
    return(0)

def convert(seconds): 
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return "%00d:%02d:%02d" % (hour, minutes, seconds) 

In [30]:
global main_log_file
WorkOn = 'FCA2_'+datetime.now().strftime("%m%d_%H%M%S")
main_log_file = WorkOn+".log" 
log_start(main_log_file,"Análise automática de arquivos de dados")


In [31]:
def csv_analysis(file):
    
    global df,ctmp, ctmp_counts, i, x  #apenas para teste, retirar na versão final

    try:
        log_write("Iniciando análise de "+file)
        
        # open the file
        if 'csv' in file:
            try:
                f = open(datain+"/"+file)
                line = f.readline()
                f.close()
                semicolon = line.count(";")
                comma = line.count(",")
                if semicolon > comma:
                    separador = ";"
                else:
                    separador = ","
                log_write("Separador de CSV selecionado ["+separador+"]")
                df = pd.read_csv(datain+"/"+file, encoding ='latin1', engine='python', sep = separador)
            except Exception as erro:
                log_write("Erro "+str(erro))
                log_write("Abortando analise "+file+"\n")
                return(-1)
        elif 'xls'in file:
            try:
                df = pd.read_excel(datain+"/"+file)
            except Exception as erro:
                log_write("Erro "+str(erro))
                log_write("Abortando analise "+file+"\n")
                return(-1)
        else:
            log_write("Erro identificando xls/csv")
            return(-1)
    
        parse_file = open(dataout+"\\"+file+'.txt',"w+")
        parse_file.write(FC_Auto_Analyser_Version+"\n") 
        parse_file.write("Arquivo "+ file +"\n" ) 
        parse_file.write(repeat_to_length("-",80) + "\n")
        
        ## MORFOLOGIA
        reg_total = df.shape[0]
        parse_file.write("### MORFOLOGIA de "+ file +"\n\n" ) 
        parse_file.write('{:4d}'.format(reg_total)+" registros e "+
                         str(df.shape[1])+" campos, sendo: "+"\n\n" ) 
        log_write(str(reg_total)+" registros e "+ str(df.shape[1])+" campos" )

        xext = '' ;  xqte = 0
        for x in df.columns:
            if df[x].dtype == np.object:
                xext = xext + sep(xqte) +x  ; xqte += 1 
        parse_file.write('{:4d}'.format(xqte)+" campos tipo texto"+"\n" ) 
        if xqte != 0: parse_file.write("     ["+xext+"]\n" ) 
        log_write(str(xqte)+" campos tipo texto" )

        xqte_corr = 0
        xext = '' ;  xqte = 0
        for x in df.columns:
            if df[x].dtype == np.int64:
                xext = xext + sep(xqte) +x  ; xqte += 1 
        parse_file.write('{:4d}'.format(xqte)+" campos tipo numérico (inteiro)"+"\n" ) 
        if xqte != 0: parse_file.write("     ["+xext+"]\n" ) 
        log_write(str(xqte)+" campos tipo numérico (inteiro)" )
        xqte_corr += xqte

        xext = '' ;  xqte = 0
        for x in df.columns:
            if df[x].dtype == np.float64:
                xext = xext + sep(xqte) +x  ; xqte += 1 
        parse_file.write('{:4d}'.format(xqte)+" campos tipo numério (decimal)"+"\n" ) 
        if xqte != 0: parse_file.write("     ["+xext+"]\n" ) 
        log_write(str(xqte)+" campos tipo numérico (decimal)" )
        xqte_corr += xqte

        xext = '' ;  xqte = 0
        for x in df.columns:
            if not (df[x].dtype == np.object or df[x].dtype == np.float64 or df[x].dtype == np.int64):
                xext = xext + sep(xqte) +x  ; xqte += 1 
        parse_file.write('{:4d}'.format(xqte)+" campos de outros tipos"+"\n" ) 
        if xqte != 0: parse_file.write("     ["+xext+"]\n" ) 
        log_write(str(xqte)+" campos de outros tipos" )
    
        ## CAMPOS TEXTO
        parse_file.write(repeat_to_length("-",80) + "\n")
        parse_file.write("### ANÁLISE DOS CAMPOS TIPO TEXTO de "+ file +"\n\n" ) 
        for x in df.columns:
            if df[x].dtype == np.object:
                xext = xext + sep(xqte) +x  ; xqte += 1 
                parse_file.write(str(xqte)+") ** "+ x + " ["+x.upper()+"] \n\n") 
                ctmp = df[x]
                ctmp_counts = ctmp.value_counts()
                ctmp_total = reg_total
                nulos = ctmp.isna().sum() 
                ctmp = ctmp.dropna()
                parse_file.write(repeat_to_length(" ",6)+"registros:  "+'{:n}'.format(ctmp_total) + " \n") 
                parse_file.write(repeat_to_length(" ",6)+"missing:    "+'{:n}'.format(nulos) + " \n") 
                parse_file.write(repeat_to_length(" ",6)+"válidos:    "+'{:n}'.format(ctmp_total-nulos) + " \n") 
                ctmp.drop_duplicates(keep='first', inplace = True) 
                ctmp_final = ctmp.shape[0]
                parse_file.write(repeat_to_length(" ",6)+"duplicados: "+'{:n}'.format(ctmp_total-nulos-ctmp_final) + " \n") 
                parse_file.write(repeat_to_length(" ",6)+"categorias: "+'{:n}'.format(ctmp_final) + " \n\n")

                if (ctmp_total-ctmp_final) == 0:
                    parse_file.write(repeat_to_length(" ",6)+"<<<categorias = registros, zero duplicados>>>\n")
                else:
                    parse_file.write(repeat_to_length(" ",6)+"Freqs  [f.abs] [ f.rel%] [f.acc%] categorias (max = "+'{:n}'.format(max_freq) +")\n")
                    freq     = 0
                    freq_acc = 0
                    for key, value in ctmp_counts.iteritems():
                        if freq <= max_freq:
                            freq += 1
                            freq_acc = freq_acc + (value/ctmp_total)
                            parse_file.write(repeat_to_length(" ",6)+"["+'{:>12,.0f}'.format(value)+
                                             "] [ " +'{:>5,.1f}'.format(value/ctmp_total*100) +"%] ["  
                                                    +'{:>5,.1f}'.format(freq_acc*100) +"%] "  
                                             +str(key)+" \n")         
                            
                                                
                parse_file.write(" \n\n") 
    
        ## CAMPOS NUMÉRICOS (INTEIROS e DECIMAIS)
        parse_file.write(repeat_to_length("-",80) + "\n")
        parse_file.write("### ANÁLISE DOS CAMPOS NUMÉRICOS (INTEIROS E DECIMAIS) de "+ file +"\n\n" ) 
        for x in df.columns:
            if df[x].dtype == np.int64 or df[x].dtype == np.float64:
                xqte += 1 
                parse_file.write(str(xqte)+") ** "+ x + " ["+x.upper()+"] ")
    
                if df[x].sum() == 0:   
                    parse_file.write("\n\n"+repeat_to_length(" ",6)+"<<<Todos os valores zerados>>>\n\n") 
                else:
                    if df[x].dtype == np.int64:
                        parse_file.write("[INTEIRO]") 
                    else:
                        parse_file.write("[DECIMAL]")
                        
                    parse_file.write("\n\n") 
                    ctmp = df[x]
                    nulos = ctmp.isna().sum() 
                    ctmpZ = ctmp[ctmp != 0]
                    ctmpZ = ctmpZ.dropna()
                    ctmpZEXC = ctmp[ctmp == 0]
                    zerados = ctmpZEXC.shape[0]
                    parse_file.write(repeat_to_length(" ",6)+"Registros:    "+'{:>15,.0f}'.format(reg_total) + "\n")
                    parse_file.write(repeat_to_length(" ",6)+"Missing:      "+'{:>15,.0f}'.format(nulos) + "\n")
                    parse_file.write(repeat_to_length(" ",6)+"Válidos:      "+'{:>15,.0f}'.format(reg_total-nulos) + "\n")
                    parse_file.write(repeat_to_length(" ",6)+"Zerados:      "+'{:>15,.0f}'.format(zerados) + "\n\n")

                    parse_file.write(repeat_to_length(" ",31)+"[Válidos]")
                    if zerados != 0: parse_file.write(repeat_to_length(" ",2)+"[Válidos Exc. Zero]"+"\n")
                    else:            parse_file.write("\n")
                                         
                                     
                    parse_file.write(repeat_to_length(" ",6)+"Registros:    "+'{:>15,.0f}'.format(reg_total-nulos))
                    if zerados != 0: parse_file.write("      "+'{:>15,.0f}'.format(ctmpZ.shape[0])+"\n" )
                    else:            parse_file.write("\n")
                                     

                    parse_file.write(repeat_to_length(" ",6)+"Soma:         "+'{:>20,.4f}'.format(ctmp.sum()))
                    if zerados != 0: parse_file.write(" "+'{:>20,.4f}'.format(ctmpZ.sum())+"\n" )
                    else:            parse_file.write("\n")
                    

                    parse_file.write(repeat_to_length(" ",6)+"Média:        "+'{:>20,.4f}'.format(ctmp.describe()[1]))
                    if zerados != 0: parse_file.write(" "+'{:>20,.4f}'.format(ctmpZ.describe()[1])+"\n" )
                    else:            parse_file.write("\n")
                    
                    parse_file.write(repeat_to_length(" ",6)+"Desvio:       "+'{:>20,.4f}'.format(ctmp.describe()[2]))
                    if zerados != 0: parse_file.write(" "+'{:>20,.4f}'.format(ctmpZ.describe()[2])+"\n" )
                    else:            parse_file.write("\n")

                    parse_file.write(repeat_to_length(" ",6)+"Mínimo:       "+'{:>20,.4f}'.format(ctmp.describe()[3]))
                    if zerados != 0: parse_file.write(" "+'{:>20,.4f}'.format(ctmpZ.describe()[3])+"\n" )
                    else:            parse_file.write("\n")

                    parse_file.write(repeat_to_length(" ",6)+"Máximo:       "+'{:>20,.4f}'.format(ctmp.describe()[7]))
                    if zerados != 0: parse_file.write(" "+'{:>20,.4f}'.format(ctmpZ.describe()[7])+"\n" )
                    else:            parse_file.write("\n")

                    parse_file.write(repeat_to_length(" ",6)+"Amplitude:    "+'{:>20,.4f}'.format(ctmp.describe()[7]-
                                                            ctmp.describe()[3]))
                    if zerados != 0: parse_file.write(" "+'{:>20,.4f}'.format(ctmpZ.describe()[7]-
                                                            ctmpZ.describe()[3])+"\n" )
                    else:            parse_file.write("\n")

                    parse_file.write(repeat_to_length(" ",6)+"25%:          "+'{:>20,.4f}'.format(ctmp.describe()[4]))
                    if zerados != 0: parse_file.write(" "+'{:>20,.4f}'.format(ctmpZ.describe()[4])+"\n" )
                    else:            parse_file.write("\n")

                    parse_file.write(repeat_to_length(" ",6)+"50%:          "+'{:>20,.4f}'.format(ctmp.describe()[5]))
                    if zerados != 0: parse_file.write(" "+'{:>20,.4f}'.format(ctmpZ.describe()[5])+"\n" )
                    else:            parse_file.write("\n")

                    parse_file.write(repeat_to_length(" ",6)+"75%:          "+'{:>20,.4f}'.format(ctmp.describe()[6]))
                    if zerados != 0: parse_file.write(" "+'{:>20,.4f}'.format(ctmpZ.describe()[6])+"\n" )
                    else:            parse_file.write("\n")


                    grf = df[x].dropna()    
                    grfINFO = grf.describe()
                    max_height = (np.histogram(grf, bins=hist_bins)[0].max())
                    max_lenght = (grfINFO[7]- grfINFO[3])
                    sns.set_style("whitegrid")
                    plt.figure(figsize=(11.7, 8.27))
                    plt.xlim(grfINFO[3].round(0), grfINFO[7].round(0))
                    plt.ylim(0, max_height*1.1)
                    
                    plt.text(0.03*max_lenght,-0.09*max_height,
                             FC_Auto_Analyser_Version+'\n'+'http://github.com/fabianocastello/fca2',
                             fontsize=10, ha='left')
                    
                    sns_plot = sns.distplot(grf, bins=hist_bins, kde=False, color="purple", 
                                 axlabel=False, rug=True)
                    sns_plot = sns_plot.set_title("histograma de ["+grf.name+"]", {'size': '18'})
                    fig = sns_plot.get_figure().savefig(dataout+"\\"+file+' HIST '+grf.name+'.png')  #pdf: trocar sufixo
                    plt.close()
                    log_write("Histograma gravado "+file+' HIST '+grf.name+'.png')

                        
                        
                    parse_file.write(" \n\n") 
                        
        parse_file.write(repeat_to_length("-",80) + "\n")
        
        ## CORRELAÇÃO ENTRE VARIÁVEIS NUMÉRICAS
        sns.set_style("whitegrid")
        plt.figure(figsize=(11.7, 8.27))
        plt.text(0.3, xqte_corr,
                 FC_Auto_Analyser_Version+'\n'+'http://github.com/fabianocastello/fca2',
                 fontsize=10, ha='left')
        sns_corr = sns.heatmap(df.corr())
        sns_corr = sns_corr.set_title("Correlação entre variáveis", {'size': '18'})
        sns_corr.get_figure().savefig(dataout+"\\"+file+' CORR.png')
        plt.close()

        ## CLOSING
        try:
            r = requests.get(readme_git)
            version_git = r.text[2:43].strip()
            if version_git != FC_Auto_Analyser_Version:
                parse_file.write("** ATENÇÃO! nova versão disponível!\n")
                parse_file.write("  Você está usando: "+FC_Auto_Analyser_Version + "\n")
                parse_file.write("  e a nova é:       "+version_git+ "\n")            
                parse_file.write("  visite github.com/fabianocastello/fca2 para baixar a nova versão \n")            
                parse_file.write(repeat_to_length("-",80) + "\n")            
            parse_file.write("READ.ME de "+readme_git+"\n\n")
            parse_file.write(r.text)
        except Exception as erro:
            parse_file.write("** Não foi possível recuperar README do github!\n")
            parse_file.write("Erro "+str(erro)+"\n")
            parse_file.write("** Leia o README no endereço\n")
            parse_file.write("   github.com/fabianocastello/fca2\n")
            
            parse_file.write(repeat_to_length("-",80) + "\n")            

        parse_file.write(repeat_to_length("-",80) + "\n")  
        parse_file.write(file+" ending. Bye!\n")
        parse_file.close() 
        log_write("Análise finalizada de "+file+ "\n")
        return(0)

    except Exception as erro:
        parse_file.write("\n\n Erro Geral: "+str(erro) + "\n\n") 
        parse_file.close() 
        log_write("Erro Geral: "+str(erro))
        log_write("Abortando analise "+file+"\n")
        return(-2)
        
def sep(ser):
    if ser > 0: return(" | ")
    return("")    

In [32]:

files = os.listdir(datain)
dfx = [i for i in files if 'XLS' in i.upper()] 
dfc = [i for i in files if 'CSV' in i.upper()] 
filesToWorkOn = dfx + dfc
log_write("Procurando arquivos em "+datain)
log_write("Arquivos tipos xls "+'{:3d}'.format(len(dfx)))
log_write("Arquivos tipos csv "+'{:3d}'.format(len(dfc)))
log_write("Total              "+'{:3d}'.format(len(filesToWorkOn))+"\n")
del dfx, dfc, files
filesToWorkOn = sorted(filesToWorkOn) 
all_files_to_log ="Arquivos para analisar:"+ "\n" + repeat_to_length(" ",30)
for name in filesToWorkOn:
    all_files_to_log = all_files_to_log + name + "\n" + repeat_to_length(" ",30)

log_write(all_files_to_log)

for name in filesToWorkOn:
    csv_analysis(name)

log_close()
          


0